In [ ]:
from odc.stac import configure_s3_access, load
from pystac_client import Client as PystacClient
import numpy as np
import odc.geo.xr


In [ ]:
from dask.distributed import Client as DaskClient

dask_client = DaskClient()
dask_client

In [ ]:
catalog = "https://explorer.dea.ga.gov.au/stac"

stac_client = PystacClient.open(catalog)

configure_s3_access(
    cloud_defaults=True,
    aws_unsigned=True,
)

In [ ]:
date_query = ['2022-10','2022-12']

collections_query = ['ga_ls8c_ard_3', 'ga_ls9c_ard_3']

filter_query = "eo:cloud_cover < 10"

#landsat 5 ['ga_ls5t_ard_3']
#landsat 8 and 9 ['ga_ls8c_ard_3', 'ga_ls9c_ard_3']

In [ ]:
items = stac_client.search(
    collections = collections_query,
    bbox= [145.910476061,-29.153361653,148.745019431,-30.128112653],
    datetime = date_query,
    filter= filter_query
).item_collection()

print(f"Found {len(items)} items")

#bbox south [145.910476061,-29.153361653,148.745019431,-30.128112653]
#bbox north [146.963236949,-27.723499515,148.806899103,-29.169637680]

In [ ]:
ds_filtered = load(
    items,
    bands=['nbart_red', 'nbart_green', 'nbart_nir',
                     'nbart_swir_1', 'nbart_swir_2'],
    crs="utm",
    chunks={"time": 27, "x": 400, "y": 400},
    resolution=30,
    groupby="solar_day",
    bbox= [145.910476061,-29.153361653,148.745019431,-30.128112653],
)

ds_filtered

In [ ]:
# Calculate the chosen vegetation proxy index and add it to the loaded data set
ds_filtered['fwi'] =1.7204 + 171*(ds_filtered.nbart_green/10000) + 3*(ds_filtered.nbart_red/10000) - 70*(ds_filtered.nbart_nir/10000) - 45*(ds_filtered.nbart_swir_1/10000)-71*(ds_filtered.nbart_swir_2/10000)

X_bin = ds_filtered['fwi'] >= -10.63 #water indez threhsold using Narran Lakes floodplain data

X_bin = X_bin.where(np.isnan(ds_filtered['fwi'])==False, np.nan)

ds_masked['water'] = X_bin

In [ ]:
# Resample using medians
print("Calculating seasonal maximum water extent...")
seasonmawater = ds_masked['water'].quantile(.95,'time').compute()

seasonmaxwater

In [ ]:
seasonmaxfwi.odc.write_cog(
    "spring22maxextent_south.tif",
    overwrite = True
)